In [3]:
import findspark
findspark.init('/home/kant/spark-2.4.4-bin-hadoop2.7')

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName("CustomerChurn").getOrCreate()

In [8]:
data = spark.read.csv('customer_churn.csv',inferSchema=True, header=True)

In [9]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [10]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [11]:
from pyspark.ml.feature import VectorAssembler

In [12]:
assember = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],outputCol='features')

In [13]:
output = assember.transform(data)

In [16]:
final_data = output.select('features','churn')

In [17]:
train_churn,test_churn = final_data.randomSplit([0.7,0.3], seed=33)

In [18]:
train_churn.describe().show()

+-------+-------------------+
|summary|              churn|
+-------+-------------------+
|  count|                633|
|   mean|0.16903633491311215|
| stddev| 0.3750803967917173|
|    min|                  0|
|    max|                  1|
+-------+-------------------+



In [19]:
test_churn.describe().show()

+-------+-------------------+
|summary|              churn|
+-------+-------------------+
|  count|                267|
|   mean|0.16104868913857678|
| stddev|0.36826613847634043|
|    min|                  0|
|    max|                  1|
+-------+-------------------+



In [20]:
from pyspark.ml.classification import LogisticRegression

In [21]:
lr_churn = LogisticRegression(labelCol='churn')

In [22]:
fitted_churn_model = lr_churn.fit(train_churn)

In [23]:
training_sum = fitted_churn_model.summary

In [24]:
training_sum.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              churn|         prediction|
+-------+-------------------+-------------------+
|  count|                633|                633|
|   mean|0.16903633491311215| 0.1263823064770932|
| stddev| 0.3750803967917173|0.33254250580558353|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



# Evaluation

In [25]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [26]:
pred_and_label = fitted_churn_model.evaluate(test_churn)

In [27]:
pred_and_label.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[27.0,8628.8,1.0,...|    0|[5.27623846619313...|[0.99491437358570...|       0.0|
|[28.0,11128.95,1....|    0|[4.12555480767802...|[0.98410229057061...|       0.0|
|[30.0,7960.64,1.0...|    1|[3.15550121954657...|[0.95912493877337...|       0.0|
|[30.0,8677.28,1.0...|    0|[3.85828114106044...|[0.97933194018604...|       0.0|
|[30.0,8874.83,0.0...|    0|[3.01571822523826...|[0.95327919361247...|       0.0|
|[30.0,10183.98,1....|    0|[2.82364677050332...|[0.94394035514988...|       0.0|
|[30.0,10960.52,1....|    0|[2.31185120255275...|[0.90985380591212...|       0.0|
|[31.0,5304.6,0.0,...|    0|[3.15486095688739...|[0.95909983034999...|       0.0|
|[31.0,8688.21,0.0...|    0|[6.53142197702076...|[0.99854518691332...|       0.0|
|[31.0,10182.6,1

In [29]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='churn')

In [30]:
auc = churn_eval.evaluate(pred_and_label.predictions)

In [31]:
auc

0.7871677740863786

# predict on brand new data

In [32]:
final_lr_model = lr_churn.fit(final_data)

In [33]:
new_customer_data = spark.read.csv('new_customers.csv', inferSchema=True, header=True)

In [34]:
new_customer_data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [35]:
test_new_customer = assember.transform(new_customer_data)

In [36]:
test_new_customer.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [37]:
final_results = final_lr_model.transform(test_new_customer)

In [39]:
final_results.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

